# 21.1 Garbage Collection

What is garbage collection? Explain it in the context of Python

### Python uses two strategies for memory allocation:

- #### Reference counting

    -  Reference counting works by counting the number of times an object is referenced by other objects in the system. When references to an object are removed, the reference count for an object is decremented. When the reference count becomes zero, the object is deallocated.
    
    
- #### Garbage collection

    -  As objects are created. they are assigned to generations, and younger generatrions are examined first
    - When the number of allocations vs. the number of deallocations is greater than a threshold the automatic garbage collector will run. 


### Manual Garbage Collection

In [4]:
import gc 
i = 0 
  
# create a cycle and on each iteration x as a dictionary 
# assigned to 1 
def create_cycle(): 
    x = { } 
    x[i+1] = x 
    print(x) 

#lists are cleared whenever a full collection or  
# collection of the highest generation (2) is run 
collected = gc.collect() # or gc.collect(2) 
print("Garbage collector: collected %d objects." % (collected) )
  
print("Creating cycles...")
for i in range(10): 
    create_cycle() 

collected = gc.collect() 
  
print("Garbage collector: collected %d objects." % (collected) )

Garbage collector: collected 34 objects.
Creating cycles...
{1: {...}}
{2: {...}}
{3: {...}}
{4: {...}}
{5: {...}}
{6: {...}}
{7: {...}}
{8: {...}}
{9: {...}}
{10: {...}}
Garbage collector: collected 10 objects.


# 21.2 Closure
What does the following program print and why?

In [1]:
increment_by_i = [lambda x: x + i for i in range(10)]

print(increment_by_i[3](4))

#not 7 from (i = 3 => 3) + 4

#correct 13 from (i = 10 => 9) + 4

13


In [6]:
## how to get 7

def create_increment_function(x):
    return lambda y: y + x

increment_by_i = [create_increment_function(i) for i in range(10)]

print(increment_by_i[3](4))

7


# 21.3 Shallow and Deep Copy
Describe the difference between a shallow copy and a deep copy. When is each appropiate, and when is neither appropiate? How is deep copy implemented?

### Note
Assigment in python does not copy, its simply a variable to a target

- ### Shallow Copy

    - Copies existing value with same reference to the origial object. So if you can the new copy, the original copy will also get changed
    
    
- ### Deep Copy

    - Copies values to a new object recursively with a new reference, so they are completetly separete objects in memory  

# 21.4 Iterators and Generators
What is the difference between an iterator and a generator?

### Iterators 

- Any object that has __iter__() and __next()__ methods

#### __iter__()

Returns the iter object itseld and is used in `for` and `in` statements

#### __next__()

Returns the next value in the iteration -- if there is no more it raises the `StopIteration` exception


In [19]:
#iterable object, implements __iter__() and __next__()
import random

class RandomIncrement:
    
    def __init__(self, limit):
        self._offset = 0.0
        self._limit = limit
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self._offset += random.random()
        
        if(self._offset > self._limit):
            raise StopIteration()
        return self._offset
    
    
    #call this to change the stop condition. Its safe to interleave this with usage of the iterator
    def increment_limit(self, incremenet_amount):
        self._limit += increment_amount
    

    
test_random_iter = RandomIncrement(3)

for i in test_random_iter:
    print(i)

0.45418645013105996
1.255423971677168
1.735888511743405
1.9437145584290187
2.0609033787092876
2.8626395112895002


### Generator

Easy way to create iterators but with the cost of less flexibilty and additional functionality 

In [18]:
def random_iterator(limit):
    
    offset = 0
    while True:
        offset += random.random()
        if (offset > limit):
            return 
        
        yield offset
        
test_random_gen = random_iterator(3)

for i in test_random_gen:
    print(i)    

0.9004998275678143
1.0946162815887486
1.2550948512644786
1.3163689361046274
2.281295954569493
2.3674986119799506
2.6477182282555685


# 21.5 @Decorator

Explain what a decorator is, with an example that shows why its useful

- Functions are first class object in Python
- Functions can be passed as arguments to other functions, and returned by functions
- Function can be defined within other functions

## @staticmethod vs @classmethod

In [ ]:
# Python program to demonstrate 
# use of class method and static method. 
from datetime import date 

class Person: 
    def __init__(self, name, age): 
        self.name = name 
        self.age = age 

    # a class method to create a Person object by birth year. 
    @classmethod
    def fromBirthYear(cls,name, year): 
        return cls(name, date.today().year - year) 
    
    # a static method to check if a Person is adult or not. 
    @staticmethod
    def isAdult(age): 
        return age > 18

person1 = Person('mayank', 21) 
person2 = Person.fromBirthYear('mayank', 1996) 

print (person1.age )
print (person2.age )

# print the result 
print (Person.isAdult(22) )




In [32]:
import timeit
import functools

def time_function(f):
    '''
    Print how long a function takes to compute
    '''
    
    begin = timeit.timeit()
    result = f()
    end = timeit.timeit()
    
    print('Function call took ' + str(end-begin)+' seconds to execute')
    
    
def ackermann(m,n):
    if m == 0:
        return n + 1
    elif n == 0:
        return ackermann(m-1,1)
    else:
        return ackermann(m-1, ackermann(m,n-1))
    
time_function(functools.partial(ackermann,3,4))

Function call took -0.0007024359983915929 seconds to execute


### With @decorator

In [45]:
import timeit
import functools

def time_function(f):
    def wrapper(*args, **kwargs):
    
        begin = timeit.timeit()
        result = f(*args, **kwargs)
        end = timeit.timeit()
    
        print('Function call with arugments {all_args} took '.format(all_args='\t'.join((str(args),str(kwargs)))) + str(end-begin)+' seconds to execute')
        
        return result
    
    return wrapper
    
    
@time_function  
def ackermann(m,n):
    if m == 0:
        return n + 1
    elif n == 0:
        return ackermann(m-1,1)
    else:
        return ackermann(m-1, ackermann(m,n-1))

    
    



In [46]:
ackermann(4,3)

Function call with arugments (0, 1)	{} took 0.0006599490025109844 seconds to execute
Function call with arugments (1, 0)	{} took -0.0009461660010856576 seconds to execute
Function call with arugments (0, 2)	{} took -0.0005510749997483799 seconds to execute
Function call with arugments (1, 1)	{} took -0.0001401190002070507 seconds to execute
Function call with arugments (2, 0)	{} took -0.000965864001045702 seconds to execute
Function call with arugments (0, 1)	{} took 0.00031154899988905527 seconds to execute
Function call with arugments (1, 0)	{} took 0.0005060979983682046 seconds to execute
Function call with arugments (0, 2)	{} took -0.003070699998716009 seconds to execute
Function call with arugments (1, 1)	{} took -0.00040331599848286714 seconds to execute
Function call with arugments (0, 3)	{} took 0.00033696399987093173 seconds to execute
Function call with arugments (1, 2)	{} took 0.0010981060004269239 seconds to execute
Function call with arugments (0, 4)	{} took 0.000729507000

Function call with arugments (2, 5)	{} took 0.0008594149985583499 seconds to execute
Function call with arugments (3, 1)	{} took -0.00040142599937098566 seconds to execute
Function call with arugments (4, 0)	{} took -0.0010126819997822167 seconds to execute
Function call with arugments (0, 1)	{} took 4.4827000238001347e-05 seconds to execute
Function call with arugments (1, 0)	{} took 0.002347613997699227 seconds to execute
Function call with arugments (0, 2)	{} took -0.00047373799861816224 seconds to execute
Function call with arugments (1, 1)	{} took 0.00037017099930380937 seconds to execute
Function call with arugments (2, 0)	{} took 0.0015117059992917348 seconds to execute
Function call with arugments (0, 1)	{} took 0.0008480329997837543 seconds to execute
Function call with arugments (1, 0)	{} took 0.00039853199996287003 seconds to execute
Function call with arugments (0, 2)	{} took -0.003194689999872935 seconds to execute
Function call with arugments (1, 1)	{} took 0.000204265999

Function call with arugments (2, 5)	{} took 0.0006338309995044256 seconds to execute
Function call with arugments (3, 1)	{} took 0.0011835200002678903 seconds to execute
Function call with arugments (0, 1)	{} took 0.00014574099986930378 seconds to execute
Function call with arugments (1, 0)	{} took 0.0011209269996470539 seconds to execute
Function call with arugments (0, 2)	{} took -0.0006279720018937951 seconds to execute
Function call with arugments (1, 1)	{} took 0.0002703539994399762 seconds to execute
Function call with arugments (2, 0)	{} took -0.00023265599884325638 seconds to execute
Function call with arugments (0, 1)	{} took 0.0009001199996419018 seconds to execute
Function call with arugments (1, 0)	{} took 0.0008275429991044803 seconds to execute
Function call with arugments (0, 2)	{} took -1.1894000635948032e-05 seconds to execute
Function call with arugments (1, 1)	{} took 0.00045587800013890956 seconds to execute
Function call with arugments (0, 3)	{} took -0.00034009800

KeyboardInterrupt: 

# 21.6 List vs Tuple

In what ways are lists and tuples similar, and in what ways are they different?

### Tuples are immutable

- You cannot change the element at index i or add/delete from a tuple, all of which are possible in a list


### Tuples are Ccontainer-friendly

- Tuples can be put into sets and used as map keys because the object cannot be changed in a hashmap so that the look up will not fail


### Tuples are faster to build and access -- have smaller memeory footprint



# 21.7 * Args and ** Kwargs

What are * args and ** kwargs? Where are they appropriate?


### Both are used to pass a variable number of arguments to a function.

### * Args

- Used to pass a variable length argument list

In [47]:
def foo(x, *args):
    print(x)
    for i in range(len(args)):
        print(str(args[i]))
        
foo(3,'elliot',[2,4],9.1)

3
elliot
[2, 4]
9.1


### ** Kwargs

- Used to pass a variable number of keyword arguments

In [49]:
def foo(x, **kwargs):
    print(x)
    for (keyword,value) in kwargs.items():
        print(keyword,value)
        
foo(3,name='elliot',arr=[2,4],flo=9.1)

3
name elliot
arr [2, 4]
flo 9.1


# 21.8 Python Code

## Lambda
- Anonymous function means that a function is without a name.

In [52]:

def multiply(x):
    return (x*x)
def add(x):
    return (x+x)

funcs = [multiply, add]
for i in range(5):
    value = list(map(lambda x: x(i), funcs))
    print(value)


[0, 0]
[1, 2]
[4, 4]
[9, 6]
[16, 8]


### collections.defualtdict()

- An unordered collection of data values that are used to store data values like a map
- Holds a key value pair
- When you call an item that does not exist, it creates a default value, that you specify as the first parameter when you make the defualtdict
- `dictionary =  collections.defualtdict(lambda : list)`

### zip()

- Map the similar index of multiple containers so that they can be used just using as single entity.
- Inverse is unzip()

### map()

- Applies a function to all the items in an input list
- map(function_to_apply, list_of_inputs)


### functools.reduce()

- Applies a rolling computation to sequential pairs of values in a list.

### heapq.nlargest()

- Returns the k largest elements from the iterable specified and satisfying the key if mentioned. 


### sum()

- Returns the sum of all the elemens in a list

# 21.9 Exception Handling

Briefly describe exception handling in Python, paying special attention to the roles played by `try`, `except`, `else`, `finally`, and `raise`. Rewrite the following programming using exceptions to make it more robust

### `Try` `Finally` Block

In [114]:
def get_value(filename,key):
    handle = open(filename)
    
    try:
        file_contents = handle.read()
        js_text = json.loads(file_contents)
        
        return js_text[key]
    
    finally:
        #prevent resource leak if there is an error parsing fiel_contents
        handle.close()

### `Try` `Except` Block

In [115]:
def get_value(filename,key):
    handle = open(filename)
    
    try:
        file_contents = handle.read()
        js_text = json.loads(file_contents)
        handle.close()
        return (True, js_text[key])
    
    except ValueError:
        #prevent resource leak if there is an error parsing fiel_contents
        handle.close()
        return (False, )

### `Try` `Except` `Finally` Block

In [116]:
def get_value(filename,key):
    handle = open(filename)
    
    try:
        file_contents = handle.read()
        js_text = json.loads(file_contents)
        handle.close()
        return (True, js_text[key])
    
    except ValueError:
        #prevent resource leak if there is an error parsing fiel_contents
        return (False, )
    finally:
        #prevent resource leak if there is an error parsing fiel_contents
        handle.close()

### Creating custom Exception

In [117]:
class ElliotException(Exception):
    def __init__(self, *args):
        Exception.__init__(self, *args)
        self.line_number = args[1]

### Built-in Exceptions

`IndexError` `FloatingPointError` `EnvironmentError` `ValueError` `NameError`

# 21.10 Scoping

Explain the rules for variable scope

## Two (non-exclusive) Possibilities:

- The variable appears in an expression
- The variable is being assigned to

### When the variable appears in the expression, Python searches for it in the following order:

1. The current function
2. Enclosing scopes (e.g. containing functions)
3. The module containing the code (also referred to as the global scope)
4. The built-in scope (e.g. open)

Note: A `NameError` is raised if none of these contain a defined variable with the given name

## Note

Arrays can be defined outside a function and be called inside a function, but varibles cannot

## Basic Scoping

In [120]:
x, y, z = 'global-x', 'global-y', 'global-z'

def basic_scoping():
    print(x) # global-x
    y = 'local-y'
    global z
    z = 'local-z'
    
basic_scoping
print(x,y,z)

global-x
global-x global-y local-z


## Inner Outer Scoping

In [122]:
def inner_outer_scoping():
    def inner1():
        print(x) # outer-x
    
    def inner2():
        x = 'inner2-x'
        print(x) # inner2-x
    
    def inner3():
        nonlocal x
        x = 'inner3-x'
        print(x) # inner3-x
        
    x = 'outer-x'
    inner1(), inner2(), inner3()
    print(x) # inner3-x
    
inner_outer_scoping()
print(x,y,z)
    

outer-x
inner2-x
inner3-x
inner3-x
global-x global-y local-z


# Function Arguments

What are position, keyword and defualt arguments to a function? What does the following program output, and why?

In [125]:
def foo(x=[]):
    x.append(1)
    return x

result = foo()
print(result)
result = foo()
print(result)

[1]
[1, 1]


## Note

- Keyword arguments must be called after positional arguments (unnamed arguments)
- When using mutable objects for arugments, make the defualt argument = `None`

In [126]:
def foo(x,y,z):
    return x * y * z

foo(1,2,3)
foo(1,y=2,z=4)
foo(1,z=8,y=3)

24